<a href="https://colab.research.google.com/github/naveedkhalid091/Learn_Agentic_AI/blob/main/step02_generative_ai_for_beginners/02(b)_updated_RAG_implementation_with_PineconeDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Implementation of RAG projects:**

For RAG projects in langchain, you need to store and retreive your data.

You need the **following environment** to set in your project.

1. Install the langchain in your project for creating flexibility in switching the chat models.
2. Firstly, you need a database for data storage and its access key.  
3. An Embedding model for vectorization of your data.
4. LLM model for conversations and its access key.

Lets Install the above environment first.

In [2]:
!pip install -U -q langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 19.9 MB/s eta 0:00:00


In [3]:
!pip install -U -q langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 3.5 MB/s eta 0:00:00


In [4]:
!pip install -U -q langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.0 MB/s eta 0:00:00


## **Gettting Access of `PINECONE` & `GEMINI` using API Keys:**

In [5]:
from google.colab import userdata
import os
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY') # Getting access of PINECONE Database
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY') # Getting access of Gemini

## **Initialization of Pinecone client:**

Initializing the `Pinecone client (pc)` is the important step as this client allows you to perform various operations such as creating indexes, inserting vectors, and executing queries.

In [6]:
from pinecone import Pinecone
pc=Pinecone()

## **Create an Index in PINECONE using above client**.

**You can optionally check the existing index name using be below code to prevent duplicates:**

* **i)** First check if the index already exist with the same choosen name.

* **ii)** Secondly create an index if the same name index is not already created.

In [7]:
# i) Checking the index name if it is already exist?

existing_indexes=[]

checking_db_indexes=pc.list_indexes()
print(existing_indexes)

for info_index in checking_db_indexes:
  existing_indexes.append(info_index.name)

print(existing_indexes)

[]
['online-rag-project', 'my-books', 'my-9th-chem-book', 'second-rag-project', 'family-structure']


In [8]:
# ii) Creation of Index
from pinecone import ServerlessSpec


index_name="my-books"


if index_name in existing_indexes:
  print(f"Index {index_name} already exist")
else:
  # PROCEED WITH INDEX CREATION
  pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

Index my-books already exist


## **Accessing the above created Index:**

- Accessing the index will help us inserting the vectors/embedded data through the below line.

In [9]:
index=pc.Index(index_name)

**Note:** PINECONE database setup is successfully completed. Now you need to setup embedding model for vectorization and chunking of your data.

You can also varify your created index into the PINECONE database by signing into your database and navigate to **`Database->Indexes`**.   

## **Select Embedding model:**

This model will first ensure that all of your data has been vectorized (converted into numbers) and ready for entering into the Pinecone database through above **`index`** variable.

The Embedding model can be selected/imported from the `langchain_google_genai` library as below:   

In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

**At this stage, the database is setup and embedding model is also selected for the vectorization of data, finally the vecotrized data will enter into the Pinecone database**

The data that need to be vectorized consist of either simple `text`, `small file` or a `large file`.

The **`simple text`** & **`small file`** will not be chunked but the **`large files`** will first went through the chunking process and then after chunking, the vectorization will be done.  

Lets run all the possiblities one by one.

## **Import PineconeVectorStore**:

- The **`PineconeVectorStore`** is a class of the LangChain framework that not only **embed your files automatically** before storing the files into vector databases but it also simplifies the process of `storing` and `retrieving` vector embeddings (the text of your file).  

- However, you can **convert text into vectors** manually through the `embed_query` method as follow:

 `vector_text=embedding_model.embed_query("Hello, I am Naveed")`
  `print(vector_text)`

But This manual effort has been eliminated by the vector store:






In [11]:
from langchain_pinecone import PineconeVectorStore

## create a vector store client
vector_store=PineconeVectorStore(index=index, embedding=embedding_model)

**While:**
- The `index` parameter tells the vector store where to store and retrieve the vector embeddings.
- The embedding parameter defines how the textual data is converted into vectors.

## 1. **Prepare Documents for the upload**

- **1.** If your document is small then follow below practice:

  - Import the Document from `langchain_core.documents`.
  - if you only wanted to upload "texts" or "small document" that don't need chunking then:
    - Create a `Document` Object which contains the link of text/file you
     wanted to store into the Database.
    - Rather then writting the manual IDs for each document, you can import the `uuid` library for generating the random and unique IDs for each document.

- **2.** If your file is **large and required chunking** then follow below steps:

  - Import a library called `pymupdf4llm`, this library will convert your pdf file into a markdown file.
  - Split the markdown file into smaller chunks by importing `MarkdownTextSplitter` from the `langchain.text_splitter`.
  - Add the chunked document into database by attaching the random generated IDs through the `uuid` with documents.  

The relevent coding these steps is given below:    

## 1. **If your document is `small` or `text only` then follow below practice**:




In [12]:


# from langchain_core.documents import Document

# document_1=Document(
   # page_content="Chemistry book",
    # metadata={"/content/Chemistry 10.pdf":"Chemistry Book"} # path of the file and its title in dictionary
# )


# put all the ducments into an array because only array is accepted while adding the documents in below step
# documents=[document_1]

In [13]:
# Add documents into Database

# from uuid import uuid4

# uuids = [str(uuid4()) for _ in range(len(documents))]

# vector_store.add_documents(documents=documents, ids=uuids)

**2. If your file is large and required chunking then follow below steps:**

In [14]:
!pip install -U -q pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 76.9 MB/s eta 0:00:00


In [17]:
# convert PDF into markdown

import pymupdf4llm

pdf_path= "/content/Chemistry 9.pdf" # This book will be deleted automatically when colab session will terminate, so you need to upload it everytime
md_text=pymupdf4llm.to_markdown(pdf_path)

Processing /content/Chemistry 9.pdf...
[                                        ] (0/152)[                                        ] (  1/152)[                                        ] (  2/152)[                                        ] (  3/152)=[=                                       ] (  4/152)[=                                       ] (  5/152)[=                                       ]

In [18]:
print(md_text)

## CHEMISTRY
# 9

###### Publisher: CARAVAN BOOK HOUSE, LAHORE


-----

**All rights (Copy right etc.) are reserved with the publisher.**
Approved by the Federal Ministry of Education (Curriculum Wing), Islamabad, according to the
National Curriculum 2006 under the National Textbook and Learning Materials Policy 2007.
**N.O.C. F.2-2/2010-Chem. Dated 2-12-2010. This book has also been published by Punjab**
Textbook Board under a print licence arrangement for free distribution in all Government
School in Punjab. No part of this book can be copied in any form especially guides, help books
etc., without the written permission of the publisher.

|Col1|CONTENTS|Col3|
|---|---|---|
|Unit 1|Fundamentals of Chemistry|1|
|Unit 2|Structure of Atoms|27|
|Unit 3|Periodic Table and Periodicity of Properties|44|
|Unit 4|Structure of Molecules|58|
|Unit 5|Physical States of Matter|75|
|Unit 6|Solutions|96|
|Unit 7|Electrochemistry|113|
|Unit 8|Chemical Reactivity|138|


Authors:


###### Dr. Jaleel Ta

In [43]:
# Split markdown text into Chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                          chunk_overlap=50,
                                          separators=["\n\n", "\n", " ", ""],
                                          )

# Split the Markdown text into documents
documents = splitter.create_documents([md_text])

In [46]:
len(documents)

773

## Add document into Database after importing `uuid` library

In [47]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['fba45c3e-ab87-4da3-9e27-e655145b116c',
 'b98209bb-0aad-4731-9e52-2c516d02987f',
 '01cc8d0d-a23c-4e13-8bff-54309f825efd',
 'f6c3e860-259f-43db-9a72-7c83d03d47d0',
 'd83ca1b2-4add-485d-b858-53b4a82080db',
 'fbe30924-f0f2-4a84-a5c8-a452c6cd11f4',
 '29ac2d56-211f-4b52-b322-c17b15596f27',
 '630cda3c-3306-4853-ac0c-f8cfc5648af1',
 'c9dae85d-32d6-4c8c-91a3-d1855d2947f2',
 '87f23077-390d-4dbd-b87d-a429abe0baa9',
 '431823ab-1790-4984-a1d1-9dd7b3b27bbb',
 '440fd599-3895-4f9a-ad80-a4a6f5353852',
 '9d3d3b11-b6f4-4540-9a25-b2a2c85142b9',
 '40565848-44a0-469b-88fd-3f93c3815120',
 '1496df78-4acd-4bf4-89fa-dcde1b5d8acf',
 '50dd4a8c-e38d-4398-b5c7-b2b4bb9beeca',
 'b0a3947c-1af4-46e8-941e-635de834e34a',
 'faaff381-9764-455a-a792-986e9768c33b',
 '51dff37c-2393-4a7b-a739-f058e02aba0f',
 '8a8eaccc-3a27-44ae-ba67-7303f48931d9',
 'a4d0ee9b-d4f9-4cc4-9520-6206f4035f14',
 '011b3b8f-c196-42c6-8ab3-f371445763b4',
 'ed1c2e8b-3f97-4d0a-8e6b-4e15e67d5b5c',
 '7dc60db2-1f65-435d-b3ef-17d8825ff503',
 '43b738af-d7e5-

## **Querying from LLM regarding the document uploaded to the vector database**:

We cannot directly ask the large language model (LLM) to get information from the vector database. Instead, we follow these steps:
 - 1. **Fetch Relevant Data from database:** First, we use a function called similarity_search to find the most relevant data from the vector database based on the user's query.
 - 2. **Combine Query and Data:** Next, we create a function that combines the user's query with the relevant data fetched from the database. This helps the LLM understand what information is needed.
 - 3. **Send to LLM:** Finally, we send the combined data (the query and the relevant database information) to the LLM. The LLM will then give a response based on this combined information.  

This approach ensures that the LLM can respond accurately, using both the query and the relevant data retrieved from the database.

In [48]:
## similarity search

query= "units"

db_result=vector_store.similarity_search(
    query,
    k=10,
    )

for res in db_result:
  print(f"content:{res.page_content}\nMetadata:{res.metadata}\n")

content:express concentration of solutions. A few of these units are discussed here.
Metadata:{}

content:express concentration of solutions. A few of these units are discussed here.
Metadata:{}

content:express concentration of solutions. A few of these units are discussed here.
Metadata:{}

content:few of these units are discussed here.
Metadata:{}

content:units, e.g., for the reaction
Metadata:{}

content:unit _is 1/12 the mass of one atom of
Metadata:{}

content:elements. The unit for
Metadata:{}

content:unit of a
Metadata:{}

content:1. Convert the following units:
Metadata:{}

content:2. Convert the following units:
Metadata:{}



In [49]:
# llm calling

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    temperature=0.9,
    max_tokens=1000,
    # other params...
)

## Define a function for final answer from LLM:

In [50]:
# define a function
def answer_to_user(query:str):
  # Vector Search
  vector_results = vector_store.similarity_search(query, k=2)
  ## invoking llm
  final_answer = llm.invoke (f'''Answer this user Query: {query}, Here are some ref to answer {vector_results}, ''')
  return final_answer

In [56]:
# calling function with query

answer=answer_to_user("How many chapters are available in chemistry book? tell me the name along with chpater numbers")
print(answer.content)

There are 8 chapters in the chemistry book. Here are the chapter names and their corresponding numbers:

*   **Unit 1:** Fundamentals of Chemistry
*   **Unit 2:** Structure of Atoms
*   **Unit 3:** Periodic Table and Periodicity of Properties
*  **Unit 4:** Structure of Molecules
*   **Unit 5:** Physical States of Matter
*   **Unit 6:** Solutions
*   **Unit 7:** Electrochemistry
*   **Unit 8:** Chemical Reactivity


In [57]:
# calling function with query

answer=answer_to_user("make 10 multiple choice questions from chapter Fundamental of Chemistry? ")
print(answer.content)

Okay, here are 10 multiple-choice questions based on the "Fundamentals of Chemistry" chapter, keeping in mind the provided document snippet focuses on the title of the chapter and doesn't provide specific content:

**Instructions:** Choose the best answer for each question.

**Questions:**

1.  Which of the following is the study of matter and its properties, as well as how matter changes?
    a) Physics
    b) Biology
    c) Chemistry
    d) Geology

2.  The basic unit of matter that cannot be further broken down by chemical means is a(n):
    a) Molecule
    b) Compound
    c) Element
    d) Mixture

3.  Which of the following is NOT a state of matter?
    a) Solid
    b) Liquid
    c) Gas
    d) Energy

4.  A substance made up of two or more elements chemically combined in a fixed ratio is a(n):
    a) Mixture
    b) Element
    c) Compound
    d) Solution

5.  Which of the following is an example of a physical property?
    a) Flammability
    b) Reactivity
    c) Density
    d) Ac